Gammapy Event Sampling Prototype
===========

Authors: Fabio Pintore and Axel Donath

This notebook sketches a prototype for event sampling with Gammapy.


Here is list of TODOs:

- debug why the total number events changes when re-executing `MapEventSample.npred_total`
- add missing docstrings and make them complete, examples of the dosctring format are in the Gammapy code base (DONE)
- add sampling of phi angle for PDF (DONE)
- compute the reconstructed positions of the events (DONE)

- add application of energy dispersion (DONE)
- compute reconstructed energy of the events (DONE)



In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, vstack

In [2]:
from gammapy.cube import MapDataset

In [3]:
from gammapy.utils.random import get_random_state
from gammapy.maps import Map, MapAxis, WcsGeom
from gammapy.cube import MapEvaluator
from gammapy.cube import MapDataset

from astropy import units as u
from astropy.table import Table
from astropy.coordinates import SkyCoord,Angle

In [4]:
from gammapy.image.models import SkyGaussian
from gammapy.spectrum.models import PowerLaw
from gammapy.cube.models import SkyModel
from gammapy.irf import PSF3D, EnergyDependentMultiGaussPSF, EnergyDispersion2D, Background3D
from gammapy.utils.energy import EnergyBounds

In [5]:
from gammapy.cube import make_psf_map, make_map_background_irf, make_edisp_map

In [6]:
from gammapy.time.models import LightCurveTableModel as LC

### Inverse CDF Sampler

In [7]:
class InverseCDFSampler:
    """Inverse CDF sampler.
    
    Parameters
    ----------
    pdf : `~`gammapy.maps.Map`
      predicted source counts  
    
    """
    def __init__(self, pdf, axis=None, random_state=0):
        """Determines a set of random numbers and calculate the cumulative distribution function"""
        self.random_state = get_random_state(random_state) 
        self.axis = axis       
        
        if axis is not None:
            self.cdf = np.cumsum(pdf, axis=self.axis)
            self.cdf /= self.cdf[:, [-1]]
        else:
            self.pdf_shape = pdf.shape  #gives the shape of the PDF array

            pdf = pdf.ravel() / pdf.sum()  #flattens the array along one axis
            self.sortindex = np.argsort(pdf, axis=None) #sorting of the elements and giving the indexes

            self.pdf = pdf[self.sortindex]  #sort the pdf array
            self.cdf = np.cumsum(self.pdf)  #evaluate the cumulative sum of the PDF array
        
    def sample_axis(self):
        """Sample along a given axis.
        """
        choice = self.random_state.uniform(high=1, size=len(self.cdf))
        
        #find the indices corresponding to this point on the CDF
        index = np.argmin(np.abs(choice.reshape(-1, 1) - self.cdf), axis=self.axis)
        
        return index + self.random_state.uniform(low=-0.5, high=0.5,
                                                 size=len(self.cdf))
    def sample(self, size):
        """Draw sample from the given PDF.
        
        Parameters
        ----------
        size : int
            Number of samples to draw.
            
        Returns
        -------
        index : tuple of `~numpy.ndarray`
            Coordinates of the drawn sample      
        """
        #pick numbers which are uniformly random over the cumulative distribution function
        choice = self.random_state.uniform(high=1, size=size)

        #find the indices corresponding to this point on the CDF
        index = np.searchsorted(self.cdf, choice)
        index = self.sortindex[index]

        # map back to multi-dimensional indexing
        index = np.unravel_index(index, self.pdf_shape) 
        index = np.vstack(index)

        index = index + self.random_state.uniform(low=-0.5, high=0.5,
                                      size=index.shape)
        return index

 ### MapEventSampler 

In [50]:
class MapEventSampler:
    """Map event sampler
    
    Parameters
    ----------
    
    npred_map : `~gammapy.maps.Map`
        Predicted number of counts map.
    background_map : `~gammapy.cube.make_map_background_irf`
        IRF background map.
    random_state : seed for the random number sampler    
    lc : `~gammapy.time.models.LightCurveTableModel`
        input light-curve of the source, given with columns labelled as "time" and "normalization" (arbitrary units): 
        the bin time HAS to be costant.
    tmin : start time of the simulation, in seconds 
    tmax : stop time of the simulation, in seconds 
    """
    
#     def __init__(self, npred_map, psf_map=None, edisp_map=None, background_map=None,
#                  random_state=0, exposure = None, lc = None):
    def __init__(self, npred_map, background_map=None,
                 random_state=0, lc = None, tmin=0, tmax=3600):
        self.random_state = get_random_state(random_state)
        self.npred_map = npred_map
#         self.psf_map = psf_map
#         self.edisp_map = edisp_map
        self.background_map = background_map
        self.lc = lc
        self.tmin=tmin
        self.tmax=tmax
        
    def npred_total(self):
        """ Calculate the total number of the simulated predicted events """       
        #return self.random_state.poisson(self.npred_map.data.sum())
        return self.random_state.poisson(np.sum(self.npred_map.data))

    def nbkgpred_total(self):
        """ Calculate the total number of the simulated background events """       
        #return self.random_state.poisson(self.npred_map.data.sum())
        return self.random_state.poisson(np.sum(self.background_map.data))
        
    def sample_npred(self):
        """ Calculate energy and coordinates of the simulated source events """       
        n_events = self.npred_total()
        
        cdf_sampler = InverseCDFSampler(self.npred_map.data, random_state=self.random_state)
        
        pix_coords = cdf_sampler.sample(n_events)
        coords = self.npred_map.geom.pix_to_coord(pix_coords[::-1])
 
        if self.lc is not None:
            start_lc = self.lc.table['time'].data[0]
            stop_lc = self.lc.table['time'].data[-1]
            if (self.tmin >= start_lc) and (self.tmax <= stop_lc):
                time_range = np.where((self.lc.table['time'].data >= self.tmin) & (self.lc.table['time'].data <= self.tmax))
                normalization = self.lc.table['normalization'].data[time_range] 
                time_sampler = InverseCDFSampler(normalization,random_state=self.random_state)
                ToA = time_sampler.sample(n_events)[0]
                
            elif ((self.tmin >= start_lc) and (self.tmax > stop_lc) and (self.tmin < stop_lc)):
                time_range = np.where((self.lc.table['time'].data >= self.tmin))
                # we assume a constant source, with a mean source normalization in the choosen interval, when tmax > stop_lc
                dt = self.lc.table['time'].data[1] - self.lc.table['time'].data[0]
                mean_norm = (1. /(stop_lc-self.tmin) * 
                    np.sum(self.lc.table['normalization'].data[time_range] * np.full(len(time_range[0]),dt)) )
                normalization = np.append(self.lc.table['normalization'].data[time_range], np.full(int(self.tmax - stop_lc), mean_norm) )
                time_sampler = InverseCDFSampler(normalization,random_state=self.random_state)
                ToA = time_sampler.sample(n_events)[0]
                
            else:
                ToA = self.tmin + self.random_state.uniform(high=(self.tmax-self.tmin), size=n_events)

        else:
            ToA = self.tmin + self.random_state.uniform(high=(self.tmax-self.tmin), size=n_events)
        
        events = Table()
        events['lon_true'] = coords[0] * u.deg
        events['lat_true'] = coords[1] * u.deg
        events['e_true'] = coords[2] * u.TeV
        events['time'] = ToA * u.s
        return events

    def sample_background(self):
        """ Calculate energy and coordinates of the simulated background events """       
        # sample from background model without applying IRFs
        n_events_bkg = self.nbkgpred_total()
        cdf_sampler = InverseCDFSampler(self.background_map.data, random_state=self.random_state)
        
        pix_coords = cdf_sampler.sample(n_events_bkg)
        coords = self.npred_map.geom.pix_to_coord(pix_coords[::-1])
        
        ToA = self.random_state.uniform(high=(self.tmax-self.tmin), size=n_events_bkg)
        
        events_bkg = Table()
        events_bkg['lon_true'] = coords[0] * u.deg
        events_bkg['lat_true'] = coords[1] * u.deg
        events_bkg['e_true'] = coords[2] * u.TeV
        events_bkg['time'] = ToA * u.s
        return events_bkg

    def sample_events(self):
        """TO DO...
        Sample all events.
        
        Returns
        -------
        
        """
        events = self.sample_npred()
        events = self.apply_psf(events)
        events = self.apply_edisp(events)

        events_bkg = self.sample_background()

        table = vstack([events, events_bkg])
        
        return table

### IRFMapSampler 

In [37]:
class IRFMapSampler:
    """Map event sampler
    
    Parameters
    ----------
    
    events : astropy table of the only source(s') events
    psf_map : `~gammapy.cube.make_psf_map`
        Psf map at the position of each event.
    edisp_map : `~gammapy.cube.make_edisp_map`
        Energy dispersion map at the position of each event.        
    """
    def __init__(self, events, psf_map=None, edisp_map=None,
                 random_state=0):
        self.random_state = get_random_state(random_state)
        self.events = events
        self.psf_map = psf_map
        self.edisp_map = edisp_map

    def sample(self):
        """ Apply PSF corrections on the coordinates of the simulated source events """       
        # apply psf to list of events
        sample_pdf = InverseCDFSampler(self.psf_map, axis=1)
        pix_coord = sample_pdf.sample_axis()
        offset = theta_axis.pix_to_coord(pix_coord)
        
        phi_angle = self.random_state.uniform(360, size=len(self.events))*u.deg
        event_positions = SkyCoord(self.events['lon_true'], self.events['lat_true'], frame='icrs', unit='deg')
        event_positions = event_positions.directional_offset_by(phi_angle,offset*u.deg)
        
        self.events['lon_true'] = event_positions.ra
        self.events['lat_true'] = event_positions.dec
        self.events['lon_true'].name = 'lon_reco'
        self.events['lat_true'].name = 'lat_reco'
        
#         return self.events
    
        """ Apply the energy dispersion for the energy of the simulated source events """       
#     def apply_edisp(self):
        if self.edisp_map is not None:
            sample_pdf_edisp = InverseCDFSampler(self.edisp_map, axis=1)
            pix_coord_edisp = sample_pdf_edisp.sample_axis()
            e_corr = migra_axis.pix_to_coord(pix_coord_edisp)

            self.events['e_true'] = self.events['e_true'] * e_corr
            self.events['e_true'].name = 'e_reco'
        
        return self.events

### Toy source model

For testing let's create a toy source model with a Gaussian morphology and a power-law spectrum:

In [38]:
#random_state = get_random_state(0)
#choice = random_state.uniform(high=1, size=10)
#index = np.argmin(choice.reshape(-1, 1))
#print(index)
#print(random_state.uniform(low=-0.5, high=0.5, size=10))
#print(index, random_state.uniform(low=-0.5, high=0.5, size=10))
#print(index + random_state.uniform(low=-0.5, high=0.5, size=10))
#index = np.argmin(np.abs(choice.reshape(-1, 1) - self.cdf), axis=axis)

In [39]:
livetime = 10 * u.hour
spatial_model = SkyGaussian("0 deg", "0 deg", sigma="0.2 deg")
spectral_model = PowerLaw(amplitude="1e-11 cm-2 s-1 TeV-1")
skymodel = SkyModel(spatial_model=spatial_model, spectral_model=spectral_model)

#Now we create a reference exposure map, we use to evaluate the model:
position = SkyCoord(0.0, 0.0, frame='galactic', unit='deg')

energy_axis = MapAxis.from_bounds(1, 100, nbin=30, unit="TeV", name="energy", interp="log")

exposure = Map.create(
    binsz=0.02,
    map_type='wcs',
    skydir=position,
    width="5 deg",
    axes=[energy_axis],
    coordsys="GAL", unit="cm2 s"
)

exposure.data = 1e10 * 1000 * np.ones(exposure.data.shape)

evaluator = MapEvaluator(model=skymodel, exposure=exposure)

npred = evaluator.compute_npred()

In [40]:
time = np.linspace(0,livetime.to('s').value,int(livetime.to('s').value))

def rate(x):
	return np.exp(-x/10000)

table = Table()

table['time'] = time
table['normalization'] = rate(time)

lcurve = LC(table)

times = lcurve.table['time'].data 
ctss = lcurve.table['normalization'].data  

light = np.vstack([times,ctss])


In [41]:
# tmin=100
# tmax=50000
# start_lc = lcurve.table['time'].data[0]
# stop_lc = lcurve.table['time'].data[-1]
# # if (tmin >= start_lc) and (tmax <= stop_lc):
# #     time_range = np.where((lcurve.table['time'].data >= tmin) & (lcurve.table['time'].data <= tmax))
# #     normalization = lcurve.table['normalization'].data[time_range] 
# #     time_sampler = InverseCDFSampler(normalization,random_state=random_state)
# #     ToA = time_sampler.sample(100)
# if (tmin >= start_lc) and (tmax > stop_lc):
#     print('ciao')
#     time_range = np.where((lcurve.table['time'].data >= tmin))
#     # we assume a constant source, with a mean source normalization in the choosen interval, when tmax > stop_lc
#     dt = lcurve.table['time'].data[1] - lcurve.table['time'].data[0]
#     mean_norm = (1. /(stop_lc-tmin) * 
#         np.sum(lcurve.table['normalization'].data[time_range] * np.full(len(time_range[0]),dt)) )
#     normalization = np.append(lcurve.table['normalization'].data[time_range], np.full(int(tmax - stop_lc), mean_norm) )
#     time_sampler = InverseCDFSampler(normalization,random_state=random_state)
#     ToA = time_sampler.sample(100)
# print(np.full(len(time_range[0]),dt))
# print(time_range,dt,mean_norm,normalization)
# print(ToA)

In [48]:
sampler = MapEventSampler(npred, random_state=0, lc=lcurve, tmin=1000, tmax=30000)
events_src=sampler.sample_npred()

IN


In [49]:
events_src

lon_true,lat_true,e_true,time
deg,deg,TeV,s
float64,float64,float64,float64
359.7333370969594,-0.07244496321415028,1.5217603637521027,19027.024970442253
359.85837797220023,0.20359207355119283,2.1419659842005205,530.250595022929
359.8918551765717,-0.019506425432173274,5.622337036174219,1315.8335074657912
359.7953553171566,-0.1186550073707349,1.7314521299420373,7252.424158766621
0.15735024730484043,0.2935878554699713,1.7356792123034828,10112.362318546835
359.9989314563643,-0.03092606310887902,1.7304144260407368,12910.548690295975
0.005488112715788418,-0.06926841577782568,1.206531807265262,4529.753642524257
359.7997714508275,0.17793342586080713,3.3898717477327427,24500.94613551266


## We apply the PSF and the EDISP map


In [15]:
## PSF map creation
filename = "$GAMMAPY_DATA/cta-1dc/caldb/data/cta/1dc/bcf/South_z20_50h/irf_file.fits"
psf_gauss = EnergyDependentMultiGaussPSF.read(filename=filename, hdu="POINT SPREAD FUNCTION")
psf_3d = psf_gauss.to_psf3d(rad=np.linspace(0, 1, 100) * u.deg)

theta_axis = MapAxis.from_bounds(0, 0.5, nbin=100, unit="deg", name="theta")

geom_psf = WcsGeom.create(
    binsz=0.2,
    skydir=position,
    width="5 deg",
    axes=[theta_axis, energy_axis],
    coordsys="GAL"
)

psf_map = make_psf_map(psf_3d, geom=geom_psf, pointing=geom_psf.center_skydir, max_offset=3 * u.deg)

events_src.sort("e_true")
coord = {
    "lon": events_src["lon_true"].reshape(-1, 1),
    "lat": events_src["lat_true"].reshape(-1, 1),
    "energy": events_src["e_true"].quantity.reshape(-1, 1),
    "theta": (theta_axis.center * theta_axis.unit)
}

pdf = psf_map.psf_map.interp_by_coord(coord)


### EDISP map creation
edisp2D = EnergyDispersion2D.read(filename=filename, hdu="ENERGY DISPERSION")
migra_axis = MapAxis.from_bounds(0, 2, nbin=30, name="migra")

geom = WcsGeom.create(
    skydir=position, binsz=0.02, width="5 deg", coordsys="GAL", axes=[migra_axis,energy_axis]
)

edisp_map = make_edisp_map(edisp=edisp2D, geom=geom, pointing=geom.center_skydir, max_offset=3 * u.deg)

events_src.sort("e_true")
coord_edisp = {
    "lon": events_src["lon_true"].reshape(-1, 1),
    "lat": events_src["lat_true"].reshape(-1, 1),
    "energy": events_src["e_true"].quantity.reshape(-1, 1),
    "migra" : (migra_axis.center * migra_axis.unit),
}

pdf_edisp=edisp_map.edisp_map.interp_by_coord(coord_edisp)

In [16]:
sampler = IRFMapSampler(events_src, psf_map=pdf, edisp_map=pdf_edisp)
#sampler = IRFMapSampler(events_src, psf_map=pdf)
events_src_irf=sampler.sample()

In [17]:
events_src_irf

lon_reco,lat_reco,e_true,time
deg,deg,TeV,s
float64,float64,float64,float64
0.07823964534958633,0.03294228583710436,1.025257281334419,13094.94203563773
359.95538030905317,0.07957931416859967,1.0311767245136272,4399.674772004161
0.1290450332134964,-0.150294014532731,1.0490503500762145,5811.407732957485
359.7254234582412,-0.3365237551944858,1.0630362412059156,12219.23988391783
359.8231999000277,0.21599295006257038,1.069196828061541,757.1003922370976
359.987940760191,-0.1176627290117236,1.0832676497369136,6616.419782810778
359.6904321476026,0.3151455912082397,1.0912896825415177,24757.91030156269
0.18212294968740264,0.08613338979481934,1.1052831552514997,3360.1719571405956


## We simulate the background 

In [27]:
from gammapy.irf import load_cta_irfs
filename = (
    "$GAMMAPY_DATA/cta-1dc/caldb/data/cta/1dc/bcf/South_z20_50h/irf_file.fits"
)
irfs = load_cta_irfs(filename)

geom = WcsGeom.create(
    skydir=position, binsz=0.02, width="5 deg", coordsys="GAL", axes=[energy_axis]
)

background = make_map_background_irf(
    pointing=position, ontime=livetime, bkg=irfs["bkg"], geom=geom
)

In [28]:
# tmin=0
# tmax=10000
# print(tmax-tmin)
# random_state=0
# random_state = get_random_state(random_state)
# random_state.uniform(high=(tmax-tmin), size=100)

In [29]:
sampler = MapEventSampler(npred, background_map=background, random_state=0, tmin=0, tmax=10000)
events_bkg=sampler.sample_background()
events_bkg

lon_true,lat_true,e_true,time
deg,deg,TeV,s
float64,float64,float64,float64
1.4712482274569896,-2.0714340758448695,1.4765230624131571,3739.7188438474295
0.6075826085921691,-0.1970144690962937,1.958995752606797,2612.7574630785134
1.3604674002218462,-0.9169917414422138,2.17224389630829,2848.585219300035
1.8089506770434782,-1.084306294719774,1.509324907230878,6863.700832688711
0.8519277516944149,-0.9029079039750186,2.235420671302714,4568.9083035790145
359.9249022999986,0.5171981274115354,1.297699776864798,3087.8556733573805
0.5070579843982214,-1.179775305503999,1.123918360061966,6493.146443584613
359.0898581980895,-2.391201140941377,2.3489223131165886,4569.463825734327


# TOTAL EVENT LIST

In [79]:
#this is not crucial...
events_bkg['lon_true'].name = 'lon_reco'
events_bkg['lat_true'].name = 'lat_reco'
events_bkg['e_true'].name = 'e_reco'

In [80]:
table = vstack([events_src_psf_edisp, events_bkg])

In [81]:
print(table)

      lon_reco            lat_reco            e_reco              time       
        deg                 deg                TeV                 s         
------------------- ------------------- ------------------ ------------------
0.07823964534958633 0.03294228583710436 1.0032358852294052  13094.94203563773
 359.95538030905317 0.07957931416859967 1.0497384536728844  4399.674772004161
 0.1290450332134964  -0.150294014532731 1.0305306964899101  5811.407732957485
  359.7254234582412 -0.3365237551944858 1.0603565780810564  12219.23988391783
  359.8231999000277 0.21599295006257038   1.01564811099223  757.1003922370976
   359.987940760191 -0.1176627290117236 1.0526586465233525  6616.419782810778
  359.6904321476026  0.3151455912082397 1.0616096537873438  24757.91030156269
0.18212294968740264 0.08613338979481934 1.2211353917370145 3360.1719571405956
  359.9123030545063 -0.4310445092018361  1.203752898860843 12298.118826168242
0.10464877751956898 -0.0383323680512579 1.1142433987322509  253.